# Trabajando con Pandas, MySQL y Google BigQuery

En este ejercicio, vamos a aprender cómo interactuar con una base de datos MySQL utilizando Pandas y luego cargar los datos en Google BigQuery. Siga las instrucciones a continuación para completar el ejercicio.

## Paso 1: Instalación de paquetes

Instale los siguientes paquetes utilizando `pip`:

1. mysql-connector-python
2. google-cloud

In [ ]:
# %pip install mysql-connector-python
# %pip install google-cloud-bigquery
# %pip install pandas
# %pip install pyarrow

## Paso 2: Importar paquetes necesarios

Importe los siguientes paquetes:

1. mysql
2. pandas
3. mysql.connector
4. google.cloud.bigquery
5. os

In [39]:
import pandas as pd
import mysql.connector as connection
from google.cloud import bigquery
from dotenv import load_dotenv
import os
load_dotenv() # necesario para utilizar el .env

True

## Paso 3: Establecer la conexión con la base de datos MySQL

Complete el siguiente código para establecer una conexión con la base de datos MySQL:

```
mydb = mysql.connector.connect(
    host="",
    user="",
    passwd="",
    database=""
)
```



In [40]:
mydb = connection.connect(
    host= os.getenv('DB_HOST'), #DB_HOST
    user= os.getenv('DB_USER'), #DB_USER
    passwd= os.getenv('DB_PASS'), #DB_PASS
    database=os.getenv('DB_DB') #DB_DB
)

## Paso 4: Leer los datos de la base de datos MySQL utilizando Pandas

Escriba el código para leer los datos de la tabla `categories` de la base de datos MySQL utilizando Pandas y almacenarlos en un DataFrame llamado `df`. No olvide cerrar la conexión a la base de datos después de realizar la consulta.


In [41]:
sql = "SELECT * FROM retail_db.categories"
df = pd.read_sql(sql, mydb)
mydb.close()
df.head()

C:\Users\GGudino\AppData\Local\Temp\ipykernel_27916\2935381959.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, mydb)


,category_id,category_department_id,category_name
0,1,2,Football
1,2,2,Soccer
2,3,2,Baseball & Softball
3,4,2,Basketball
4,5,2,Lacrosse


## Paso 5: Configurar las credenciales de Google Cloud

Agregue el archivo JSON de las credenciales de su cuenta de Google Cloud al entorno de ejecución y establezca la variable de entorno `GOOGLE_APPLICATION_CREDENTIALS` con la ruta al archivo JSON.


In [42]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service.json" 

## Paso 6: Crear un cliente de BigQuery

Escriba el código para crear un objeto cliente de BigQuery.


In [43]:
# client = bigquery.Client()
# Si usamos el archivo de servicio:
client = bigquery.Client.from_service_account_json("service.json")

## Paso 7: Escribir los datos del DataFrame en Google BigQuery

Escriba el código necesario para escribir los datos del DataFrame `df` en la tabla `ar_dp_dtkl_raw.categories` en Google BigQuery. Utilice la opción "WRITE_TRUNCATE" para sobrescribir cualquier dato existente en la tabla.


In [44]:
tabla_id = "acquired-script-391200.ar_dp_dtkl_raw.categories"
tabla_property = client.get_table(tabla_id)

In [45]:
job_configuration = bigquery.LoadJobConfig(
    schema = tabla_property.schema,
    write_disposition = "WRITE_TRUNCATE" # alternativas: WRITE_APPEND (agrega a la cola), WRITE_EMPTY (solo carga si esta vacía)
)

In [46]:
job = client.load_table_from_dataframe(
    df, tabla_id, job_config=job_configuration
)

job.result() #esperar a que se complete la carga
print("Carga completada")

Carga completada
